In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip uninstall -y bitsandbytes triton peft transformers accelerate torch
!pip install \
  torch==2.2.2 \
  transformers==4.41.0 \
  peft==0.10.0 \
  accelerate==0.30.1 \
  bitsandbytes==0.43.1 \
  triton==2.2.0

Found existing installation: bitsandbytes 0.43.1
Uninstalling bitsandbytes-0.43.1:
  Successfully uninstalled bitsandbytes-0.43.1
Found existing installation: triton 2.2.0
Uninstalling triton-2.2.0:
  Successfully uninstalled triton-2.2.0
Found existing installation: peft 0.10.0
Uninstalling peft-0.10.0:
  Successfully uninstalled peft-0.10.0
Found existing installation: transformers 4.41.0
Uninstalling transformers-4.41.0:
  Successfully uninstalled transformers-4.41.0
Found existing installation: accelerate 0.30.1
Uninstalling accelerate-0.30.1:
  Successfully uninstalled accelerate-0.30.1
Found existing installation: torch 2.2.2
Uninstalling torch-2.2.2:
  Successfully uninstalled torch-2.2.2
  Using cached torch-2.2.2-cp312-cp312-manylinux1_x86_64.whl.metadata (25 kB)
  Using cached transformers-4.41.0-py3-none-any.whl.metadata (43 kB)
  Using cached peft-0.10.0-py3-none-any.whl.metadata (13 kB)
  Using cached accelerate-0.30.1-py3-none-any.whl.metadata (18 kB)
  Using cached bitsa

In [3]:
import torch, triton, bitsandbytes, transformers, peft

print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Triton:", triton.__version__)
print("Transformers:", transformers.__version__)
print("PEFT:", peft.__version__)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.12/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.12/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.12/dist-package

Torch: 2.2.2+cu121
CUDA available: True
Triton: 2.2.0
Transformers: 4.41.0
PEFT: 0.10.0


In [7]:
import json

config_path = "/content/drive/MyDrive/phi3_lora_final/adapter_config.json"

with open(config_path, "r") as f:
    config = json.load(f)

config.keys()


dict_keys(['alpha_pattern', 'auto_mapping', 'base_model_name_or_path', 'bias', 'ensure_weight_tying', 'eva_config', 'exclude_modules', 'fan_in_fan_out', 'inference_mode', 'init_lora_weights', 'layer_replication', 'layers_pattern', 'layers_to_transform', 'loftq_config', 'lora_alpha', 'lora_bias', 'lora_dropout', 'megatron_config', 'megatron_core', 'modules_to_save', 'peft_type', 'peft_version', 'qalora_group_size', 'r', 'rank_pattern', 'revision', 'target_modules', 'target_parameters', 'task_type', 'trainable_token_indices', 'use_dora', 'use_qalora', 'use_rslora'])

In [6]:
import json

config_path = "/content/drive/MyDrive/phi3_lora_final/adapter_config.json"

with open(config_path, "r") as f:
    config = json.load(f)

ALLOWED_KEYS = {
    "base_model_name_or_path",
    "bias",
    "fan_in_fan_out",
    "inference_mode",
    "init_lora_weights",
    "lora_alpha",
    "lora_dropout",
    "modules_to_save",
    "peft_type",
    "r",
    "revision",
    "target_modules",
    "task_type",
}

cleaned = {k: v for k, v in config.items() if k in ALLOWED_KEYS}

with open(config_path, "w") as f:
    json.dump(cleaned, f, indent=2)

print("✅ adapter_config.json cleaned for PEFT 0.10.0")


✅ adapter_config.json cleaned for PEFT 0.10.0


In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

BASE_MODEL_ID = "microsoft/Phi-3-mini-4k-instruct"
LORA_PATH = "/content/drive/MyDrive/phi3_lora_final"

device = "cuda"
print(f"✅ Using device: {device}")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL_ID,
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    attn_implementation="eager"   # avoids flash + cache bugs
)

model = PeftModel.from_pretrained(
    base_model,
    LORA_PATH,
    is_trainable=False
)

model.config.use_cache = False
model.eval()

print("✅ Phi-3 + LoRA loaded successfully (4-bit, GPU)")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


✅ Using device: cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Phi-3 + LoRA loaded successfully (4-bit, GPU)


In [8]:
prompt = """### Instruction:
Explain why LoRA is preferred over full fine-tuning for large language models.

### Response:
"""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=120,
        do_sample=False,
        temperature=0.0,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

print(tokenizer.decode(output[0], skip_special_tokens=True))

/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


### Instruction:
Explain why LoRA is preferred over full fine-tuning for large language models.

### Response:
LoRA is preferred over full fine-tuning for large language models because it is more efficient and requires less computational resources. LoRA only requires a small number of parameters to be fine-tuned, which makes it much faster and more cost-effective than full fine-tuning. Additionally, LoRA can be used to quickly adapt a pre-trained model to a new task without having to retrain the entire model from scratch.
